In [1]:
import cv2, os, sys, numpy as np, torch, json
from tqdm.notebook import tqdm
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import math
from numpy.linalg import norm, inv
import math, torch
from numpy import cross, tan

def normalize(v):
    return v / norm(v)

In [2]:
def camera_pose(eye, front, up):
    # print('eye', eye)
    # print('front', front)
    # print('up', up)
    z = normalize(-1 * front) # -1 except for mesh
    x = normalize(cross(up, z))
    y = normalize(cross(z, x))
 
    # convert to col vector
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    z = z.reshape(-1, 1)
    eye = eye.reshape(-1, 1)

    pose = np.block([
        [x, y, z, eye],
        [0, 0, 0, 1]
    ])
    return pose

In [3]:
def sample_spherical(radius_min=1.5, radius_max=2.0, maxz=1.6, minz=-0.75):
    correct = False
    while not correct:
        vec = np.random.uniform(-1, 1, 3)
#         vec[2] = np.abs(vec[2])
        radius = np.random.uniform(radius_min, radius_max, 1)
        vec = vec / np.linalg.norm(vec, axis=0) * radius[0]
        if maxz > vec[2] > minz:
            correct = True
    return vec

In [4]:
dataset_root = '/proj/vondrick3/sruthi/zero123-internal-test/sjc/data/nerf_wild/pillowsquish'
raw_img_path = '/proj/vondrick3/sruthi/zero123-internal-test/sjc/data/nerf_wild/pillow'

In [5]:
imgs = os.listdir(raw_img_path)

In [8]:
radius = 2.0
up = np.array([0, 0, 1])

In [9]:
# polar_angles = [110., 70., 70, 90, 90, 75, 70, 80, 75, 80] # nerf_wild
# polar_angles = [90., 70., 80, 80, 85, 90,  45, 80] # nerf_object
# polar_angles = [90., 90., 90, 90, 90, 90, 90, 90, 90, 90] # nerf_sculpture
# polar_angles = [50., 60., 90, 90, 90, 90, 80, 80, 75, 80] # nerf_art
# polar_angles = [85, 90, 90, 90] # nerf_art
polar_angles = [90 for i in imgs] # nerf_art
fov = math.radians(49.1)

In [10]:
for i_img in range(len(imgs)):
    scene_name = i_img
    scene_dir = os.path.join(dataset_root, str(scene_name))
    os.makedirs(scene_dir, exist_ok=True)
    
    # move images
    img_dir = os.path.join(raw_img_path, '%d.png' % i_img)
    img = imageio.imread(img_dir)
    img = cv2.resize(img, [800, 800], interpolation = cv2.INTER_LINEAR)
    img = Image.fromarray(img)
    train_dir = os.path.join(scene_dir, 'train')
    os.makedirs(train_dir, exist_ok=True)
    new_img_dir = os.path.join(train_dir, 'r_0.png')
    img.save(new_img_dir)
    
    # create transforms
    angle = polar_angles[i_img]
    eye = normalize(np.array([0, math.tan(math.radians(angle)), 1])) * radius
    c2w = camera_pose(eye, -eye, up)
    
    transformations = {}
    transformations["camera_angle_x"] = fov
    transformations["frames"] = []
    frame = {}
    frame["file_path"] = './' + '/'.join(new_img_dir.split('/')[-2:])[:-4]
    frame["transform_matrix"] = c2w.tolist()
    transformations["frames"].append(frame.copy())
    
    for i in range(32):
        eye = sample_spherical(1.9, 2.1, 2.1, -1.0)
        c2w = camera_pose(eye, -eye, up)
        frame['transform_matrix'] = c2w.tolist()
        transformations["frames"].append(frame.copy())
    
    json_path = os.path.join(scene_dir, 'transforms_train.json')
    with open(json_path, 'w') as f:
        json.dump(transformations, f, indent=4)

In [11]:
camera_pose(np.array([1, 0, 0]), np.array([1, 0, 0]), np.array([1, 0, 0]))

/tmp/ipykernel_3649962/4279138343.py:12: RuntimeWarning: invalid value encountered in divide
  return v / norm(v)


array([[nan, nan, -1.,  1.],
       [nan, nan,  0.,  0.],
       [nan, nan,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])